In [64]:
import warnings
warnings.filterwarnings(action='ignore')

In [65]:
# !pip install langchain langchain-core langchain-community pydantic duckduckgo-search langchain_experimental

In [66]:
# !pip install -U ddgs

## Built-in Tool - DuckDuckGo Search

In [67]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('Latest news in india today related with cricket')

print(results)

Save for later . Share. More. India Today . Indian Men’s Cricket Team Schedule 2026: Full Fixtures, Series Dates, and Tournament Breakdown. Series: South Africa tour of India , 2025. Venue: ACA-VDCA Cricket Stadium, Visakhapatnam. Date & Time: Sat, Dec 6, 1:30 PM LOCAL. Info Live Scorecard Squads Overs Highlights Full Commentary News . Cricket News .This boy has been hammering everybody across all kinds of matches. I had told last year also that they should fast-track him for the T20 World Cup. Maybe it is too late for that now, but they still fast-track him into the team. Check out the latest news from India and around the world. Latest India news on Bollywood, Politics, Business, Cricket , Technology and Travel. Latest Cricket Score News on Cricketnmore. Select Language. South Africa tour of England, 2025 The Ashes, 2025-26 Big Bash League 2025-26 International League T20, 2025-26 Sri Lanka Women tour of India 2025 View Upcoming Series.


In [68]:
print(search_tool.name)
print(search_tool.description)
print(search_tool.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool - Shell Tool

In [69]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('dir')

print(results)

Executing command:
 dir
 Volume in drive G is Drive D
 Volume Serial Number is 94D2-FC99

 Directory of g:\GenAI - CampusX\13-Tools_Langchain

27-12-2025  10:54 AM    <DIR>          .
27-12-2025  10:53 AM    <DIR>          ..
27-12-2025  01:43 PM            12,271 tools_in_langchain.ipynb
               1 File(s)         12,271 bytes
               2 Dir(s)  73,673,875,456 bytes free



In [70]:
print(shell_tool.name)
print(shell_tool.description)
print(shell_tool.args)

terminal
Run shell commands on this Windows machine.
{'commands': {'anyOf': [{'type': 'string'}, {'items': {'type': 'string'}, 'type': 'array'}], 'description': 'List of shell commands to run. Deserialized using json.loads', 'title': 'Commands'}}


## Custom Tools

In [71]:
from langchain_core.tools import tool

In [72]:
# Step 1 - create a function
def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [73]:
# Step 2 - add type hints
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [74]:
# Step 3 - add tool decorator
@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [75]:
result = multiply.invoke({"a":3, "b":5})

In [76]:
print(f"Multiplication Result : {result}")

Multiplication Result : 15


In [77]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [78]:
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


## Method 2 - Using StructuredTool

In [79]:
from langchain_core.tools import StructuredTool
from pydantic import BaseModel, Field

In [80]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [81]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [82]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [83]:
try:
    result = multiply_tool.invoke({'a':3, 'b':3})
    print(result)
    print(multiply_tool.name)
    print(multiply_tool.description)
    print(multiply_tool.args)
except Exception as e:
    print(e)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Method 3 - Using BaseTool Class

In [84]:
from langchain.tools import BaseTool
from typing import Type

In [85]:
# arg schema using pydantic
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

In [86]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"
    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [87]:
multiply_tool = MultiplyTool()

In [88]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


## Toolkit

In [89]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [90]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [91]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers
